In [5]:
import csv
import tarfile
import pandas as pd
from pathlib import Path

In [4]:
with open('data/data_good.txt') as file:
    reader = csv.reader(file, delimiter='\t')
    with open('processed/good.csv', 'w') as target_file:
        for row in reader:
            writer = csv.writer(target_file)
            writer.writerow(row)

with open('data/Total_old.txt') as file:
    reader = csv.reader(file, delimiter='\t')
    with open('processed/total.csv', 'w') as target_file:
        for row in reader:
            writer = csv.writer(target_file)
            writer.writerow(row)

In [9]:
with tarfile.open("bc.tgz") as tar:
    for item in tar:
        print(item.name[:-4])



data_good
Total_old


In [19]:
data_dir = "data"

Path(data_dir).mkdir(parents=True, exist_ok=True)

tar = tarfile.open("bc.tgz")
tar.extractall(data_dir)
for item in tar:
    extracted = open("{}/{}".format(data_dir, item.name))
    reader = csv.reader(extracted, delimiter="\t")
    target = open("{}/{}.csv".format(data_dir, item.name[:-4]), "w")
    for row in reader:
        writer = csv.writer(target)
        writer.writerow(row)